In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model, Sequential
from keras.layers import Activation, Dense,Flatten, Dropout, Input
from keras.layers import MaxPooling2D, Conv2D, AveragePooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
import keras.backend as K
import keras
from keras.losses import categorical_crossentropy

Using TensorFlow backend.


In [2]:
lmbda = 0.1
batch_size =128
epochs = 200
seed = 0
learning_rate = 0.1
data_augmentation = False
cutout = 16
budget = 0.3

## model

In [3]:
VGG13 = [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

input_shape = (32, 32, 3)
num_classes = 10
name = 1

input_tensor = Input(shape=input_shape, name='input')
net = input_tensor

for x in VGG13:
    if x == 'M':
        net = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool'+str(name))(net)
        name += 1
    else:
        net = ZeroPadding2D(padding=(1, 1), name='zero'+str(name))(net)
        net = Conv2D(64, (3, 3), padding='same', name='conv'+str(name))(net)
        net = BatchNormalization(name='norm'+str(name))(net)
        net = Activation('relu', name='acti'+str(name))(net)
        name += 1
        
net = AveragePooling2D(pool_size=(1, 1), strides=(1,1), padding='same', name='avepool')(net)
net = Flatten(name='flatten')(net)
net = Dense(512, name='dense')(net)
prediction = Dense(num_classes, name='prediction')(net)
confidence = Dense(1, name='confidence')(net)

model = Model(input_tensor, (prediction, confidence))

 ## dataset

In [4]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

## preprocess images

In [5]:
def preprocess(images, length, train=True):
    
    # Normalize
    mean = [125.3, 123.0, 113.9]
    std = [63.0, 62.1, 66.7]
    images = ((images - mean) / std)
    
    # Cutout
    if train:
        h = images[0].shape[0]
        w = images[0].shape[1]
        mask = np.ones((h, w), np.float32)
        y = np.random.randint(h)
        x = np.random.randint(w)
        y1 = int(np.clip(y - length / 2, 0, h))
        y2 = int(np.clip(y + length / 2, 0, h))
        x1 = int(np.clip(x - length / 2, 0, w))
        x2 = int(np.clip(x + length / 2, 0, w))
        mask[y1: y2, x1: x2] = 0.
        tensor = np.append(mask, mask)
        tensor = np.append(tensor, mask)
        tensor = tensor.reshape((32, 32, 3))
        images = images * tensor
        
    return images

In [6]:
x_train = preprocess(x_train, cutout, True)
x_test = preprocess(x_test, cutout, False)

In [7]:
y_train = keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes=num_classes)

## loss

In [23]:
def compute_loss(y_true, y_pred):
    # 順伝播の出力
    input = K.placeholder(shape=(None, 32, 32, 3))
    prediction = K.variable(y_pred[0])
    confidence = K.variable(y_pred[1])
    
    # clipメソッドでインプットを範囲内に収める
    eps = 1e-12
    pred_original = K.clip(prediction, 0. + eps, 1. - eps)
    confidence = K.clip(confidence, 0. + eps, 1. - eps)
    
    # 予測値の補正を行う（ヒント部分）
    # Randomly set half of the confidences to 1 (i.e. no hints)
    b = K.variable(np.random.randint(0,2,confidence.shape))
    # confを設定
    conf = confidence * b + (1 - b)
    origin_tensor = K.variable(K.get_value(conf))
    
    return origin_tensor

SyntaxError: invalid syntax (<ipython-input-23-6fd779342a26>, line 3)

In [ ]:
    
    
    origin = origin_tensor
    for i in range(K.get_value(pred_original).shape[1]-1):
        tensor = np.append(origin, origin_tensor, axis=1)
        origin  = tensor
    conf = K.variable(tensor)
    # 予測を小さくして、正解ラベルの分布を足す
    pred_new = pred_original * conf + y_true * (1 - conf)
    pred_new = K.log(pred_new)
    
    # 損失計算
    xentropy_loss = categorical_crossentropy(pred_new, K.variable(y_true))
    confidence_loss = K.mean(-K.log(confidence))
    
    # 損失を定義
    total_loss = xentropy_loss + (lmbda * confidence_loss)
    if budget > K.get_value(confidence_loss):
        lmbda = lmbda / 1.01
    elif budget <= K.get_value(confidence_loss):
        lmbda = lmbda / 0.99
        
    return total_loss

## optimizer, scheduler

In [9]:
optim = keras.optimizers.SGD(lr=learning_rate, momentum=0.9, decay=5e-4, nesterov=True)

def schedule(epoch, decay=0.2):
    if epoch in [60, 120, 160]:
        return learning_rate * decay

callbacks = [keras.callbacks.LearningRateScheduler(schedule)]

In [22]:
model.compile(optimizer=optim, loss=compute_loss)

InvalidArgumentError: You must feed a value for placeholder tensor 'input' with dtype float and shape [?,32,32,3]
	 [[Node: input = Placeholder[dtype=DT_FLOAT, shape=[?,32,32,3], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'input', defined at:
  File "/home/takano/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/takano/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/takano/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/takano/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/takano/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.sta

## Train

In [ ]:
history = model.fit(x_train, y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, verbose=1, 
                    callbacks=callbacks, 
                    validation_data=(x_test, y_test))